# Narupa Multiplayer Selections Test

This notebook demonstrates how to create a selection via the multiplayer key value store.

In [1]:
from narupa.app.client import NarupaClient
from google.protobuf.struct_pb2 import Value, Struct
from google.protobuf.json_format import MessageToDict

## Start a Narupa IMD client

In [2]:
client = NarupaClient()
first_frame = client.wait_until_first_frame()
print(first_frame.particle_count)

403


In [3]:
client.join_multiplayer("selections test")
multiplayer_client = client._multiplayer_client
player_id = multiplayer_client.player_id
print(player_id)

1


In [4]:
selection = client.create_selection("my nice selection :)", [0, 1, 2, 3, 4])

In [5]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

my nice selection :) {0.0, 1.0, 2.0, 3.0, 4.0}


In [6]:
selection.clear_particles()
selection.select_particles([5, 4, 3])
client.update_selection(selection)

In [7]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

my nice selection :) {3.0, 4.0, 5.0}
